In [1]:
import git, os, sys
git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
os.chdir(f'{git_root}/src')
sys.path.append(os.path.abspath(os.path.join(f'{git_root}/src')))
print(f'Changed working directory to {os.getcwd()}')

Changed working directory to C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src


In [2]:
#getting dependencies
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' (disables SettingWithCopyWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
import itertools

from fight_stat_helpers import *

from data_handler import DataHandler
dh = DataHandler()

C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\fight_stat_helpers.py:15: DtypeWarning: Columns (8,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  fight_hist = pd.read_csv('content/data/processed/ufc_fights.csv')
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\fight_stat_helpers.py:17: DtypeWarning: Columns (15,37,38,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  ufc_fights_crap = pd.read_csv('content/data/processed/ufc_fights_crap.csv')
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:86: DtypeWarning: Columns (15,34) have mixed types. Specify dtype option on import or set low_memory=False.
  self.csv_data = {key : pd.read_csv(self.csv_filepaths[key], sep=',') for key in self.csv_filepaths.keys()}
C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src\data_handler\data_handler.py:86: DtypeWarning: Columns (11) have mixed types. S

In [3]:
from sklearn.metrics import get_scorer_names

In [4]:
print(get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'd2_absolute_error_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_max_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples'

In [3]:
#scores a model
def model_score(dataframe, features, iloc_val = 3200, _max_iter = 2000, scoring='neg_log_loss', scaled=True):
    yyy=dataframe['result'].iloc[0:iloc_val]
    XXX=dataframe[features].iloc[0:iloc_val]
    XXXscaler = preprocessing.StandardScaler().fit(XXX)
    XXX_scaled = XXXscaler.transform(XXX) 
    X = XXX_scaled if scaled else XXX
    winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=_max_iter, fit_intercept=False)
    # find the cross val score with log loss
    return cross_val_score(winPredictionModel,X,yyy,cv=4,scoring=scoring).mean()
    
#CODE FOR THE GREEDY ALGORITHM FOR FEATURE SELECTION
def greedy(dataframe, features, subsetsize, iloc_val=3200, _max_iter = 2000, scaled=True, scoring='neg_log_loss', set_of_sets=False):
    if set_of_sets:
        s=set([tuple(feature) for feature in features])
    else:
        s=set(features)
    subsets=list(map(set, itertools.combinations(s, subsetsize))) #subsets of size (subsetsize)
    scores_dict = {}
    for subset in subsets:
        if set_of_sets:
            list_of_features = []
            for feature_set in subset:
                list_of_features.extend(list(feature_set))
        else:
            list_of_features = list(subset)
        scores_dict[tuple(subset)]=model_score(dataframe, list_of_features, iloc_val, _max_iter, scaled=scaled, scoring=scoring)
    max_key = max(scores_dict, key=scores_dict.get)
    max_score = scores_dict[max_key]
    print(f'best subset: {max_key}')
    print(f'with score {max_score}')
    best_feature_list = []
    if set_of_sets:
        for feature_set in max_key:
            best_feature_list.extend(list(feature_set))
    else:
        best_feature_list = list(max_key)
    return best_feature_list, max_score

In [4]:
ufc_fights = dh.get('ufc_fights')
ufc_fights_winner = dh.clean_ufc_fights_for_winner_prediction(ufc_fights)

In [7]:
for col in ufc_fights_winner.columns:
    print(col)

date
division
fighter
opponent
result
method
fighter_wins
fighter_losses
fighter_age
fighter_height
fighter_reach
fighter_L5Y_wins
fighter_L5Y_losses
fighter_L2Y_wins
fighter_L2Y_losses
fighter_ko_wins
fighter_ko_losses
fighter_L5Y_ko_wins
fighter_L5Y_ko_losses
fighter_L2Y_ko_wins
fighter_L2Y_ko_losses
fighter_sub_wins
fighter_sub_losses
fighter_L5Y_sub_wins
fighter_L5Y_sub_losses
fighter_L2Y_sub_wins
fighter_L2Y_sub_losses
fighter_inf_knockdowns_avg
fighter_inf_pass_avg
fighter_inf_reversals_avg
fighter_inf_sub_attempts_avg
fighter_inf_takedowns_landed_avg
fighter_inf_takedowns_attempts_avg
fighter_inf_sig_strikes_landed_avg
fighter_inf_sig_strikes_attempts_avg
fighter_inf_total_strikes_landed_avg
fighter_inf_total_strikes_attempts_avg
fighter_inf_head_strikes_landed_avg
fighter_inf_head_strikes_attempts_avg
fighter_inf_body_strikes_landed_avg
fighter_inf_body_strikes_attempts_avg
fighter_inf_leg_strikes_landed_avg
fighter_inf_leg_strikes_attempts_avg
fighter_inf_distance_strikes_land

In [5]:
# stats classes to test 
record_features = [
  u'wins',
  u'losses',
  u'L5Y_wins',
  u'L5Y_losses',
  u'L2Y_wins',
  u'L2Y_losses',
  u'ko_wins',
  u'ko_losses',
  u'L5Y_ko_wins',
  u'L5Y_ko_losses',
  u'L2Y_ko_wins',
  u'L2Y_ko_losses',
  u'sub_wins',
  u'sub_losses',
  u'L5Y_sub_wins',
  u'L5Y_sub_losses',
  u'L2Y_sub_wins',
  u'L2Y_sub_losses',
]
record_statistics=[
  u'fighter_wins',
  u'fighter_losses',
  u'fighter_L5Y_wins',
  u'fighter_L5Y_losses',
  u'fighter_L2Y_wins',
  u'fighter_L2Y_losses',
  u'fighter_ko_wins',
  u'fighter_ko_losses',
  u'fighter_L5Y_ko_wins',
  u'fighter_L5Y_ko_losses',
  u'fighter_L2Y_ko_wins',
  u'fighter_L2Y_ko_losses',
  u'fighter_sub_wins',
  u'fighter_sub_losses',
  u'fighter_L5Y_sub_wins',
  u'fighter_L5Y_sub_losses',
  u'fighter_L2Y_sub_wins',
  u'fighter_L2Y_sub_losses',
  u'opponent_wins',
  u'opponent_losses',
  u'opponent_L5Y_wins',
  u'opponent_L5Y_losses',
  u'opponent_L2Y_wins',
  u'opponent_L2Y_losses',
  u'opponent_ko_wins',
  u'opponent_ko_losses',
  u'opponent_L5Y_ko_wins',
  u'opponent_L5Y_ko_losses',
  u'opponent_L2Y_ko_wins',
  u'opponent_L2Y_ko_losses',
  u'opponent_sub_wins',
  u'opponent_sub_losses',
  u'opponent_L5Y_sub_wins',
  u'opponent_L5Y_sub_losses',
  u'opponent_L2Y_sub_wins',
  u'opponent_L2Y_sub_losses'
  ]
physical_stats=[ u'fighter_age',u'fighter_height',u'fighter_reach',u'opponent_age',u'opponent_height',u'opponent_reach']
# removed pass from punch_statistics as it is sometimes nan
event_features = [
                  u'knockdowns',
                #   u'pass',
                  u'reversals',
                  u'sub_attempts',
]

punch_features = [
                  u'takedowns',
                  u'sig_strikes',
                  u'total_strikes',
                  u'head_strikes',
                  u'body_strikes',
                  u'leg_strikes',
                  u'distance_strikes',
                  u'clinch_strikes',
                  u'ground_strikes',
              ]


def make_event_statistics_subset(event_list):
    event_statistics = []
    for event in event_list:
        event_statistics.append(f'fighter_inf_{event}_avg')
        event_statistics.append(f'fighter_abs_{event}_avg')
        event_statistics.append(f'opponent_inf_{event}_avg')
        event_statistics.append(f'opponent_abs_{event}_avg')
    return event_statistics
  
def make_event_statistics_subset_diff(event_list):
    event_statistics = []
    for event in event_list:
        event_statistics.append(f'fighter_inf_{event}_avg_diff_2')
        event_statistics.append(f'fighter_abs_{event}_avg_diff_2')
    return event_statistics

def make_punch_statistics_subset(punch_stat_list):
    punch_statistics = []
    for punch in punch_stat_list:
        punch_statistics.append(f'fighter_inf_{punch}_landed_avg')
        punch_statistics.append(f'fighter_inf_{punch}_attempts_avg')
        punch_statistics.append(f'fighter_abs_{punch}_landed_avg')
        punch_statistics.append(f'fighter_abs_{punch}_attempts_avg')
        punch_statistics.append(f'opponent_inf_{punch}_landed_avg')
        punch_statistics.append(f'opponent_inf_{punch}_attempts_avg')
        punch_statistics.append(f'opponent_abs_{punch}_landed_avg')
        punch_statistics.append(f'opponent_abs_{punch}_attempts_avg')
    return punch_statistics
  
def make_punch_statistics_subset_diff(punch_stat_list):
    punch_statistics = []
    for punch in punch_stat_list:
        punch_statistics.append(f'fighter_inf_{punch}_landed_avg_diff_2')
        punch_statistics.append(f'fighter_inf_{punch}_attempts_avg_diff_2')
        punch_statistics.append(f'fighter_abs_{punch}_landed_avg_diff_2')
        punch_statistics.append(f'fighter_abs_{punch}_attempts_avg_diff_2')
    return punch_statistics

def make_punch_statistics_subset_diff_set_of_sets(punch_stat_list):
    punch_statistics = []
    for punch in punch_stat_list:
        punch_statistics.append([f'fighter_inf_{punch}_landed_avg_diff_2',
        f'fighter_inf_{punch}_attempts_avg_diff_2',
        f'fighter_abs_{punch}_landed_avg_diff_2',
        f'fighter_abs_{punch}_attempts_avg_diff_2',])
    return punch_statistics
  
punch_statistics = make_punch_statistics_subset(punch_features)
event_statistics = make_event_statistics_subset(event_features)
punch_statistics_diff = make_punch_statistics_subset_diff(punch_features)
event_statistics_diff = make_event_statistics_subset_diff(event_features)
punch_statistics_diff_set_of_sets = make_punch_statistics_subset_diff_set_of_sets(punch_features)

record_statistics_diff = [record_statistics[i]+'_diff_2' for i in range(len(record_statistics)//2)]
physical_stats_diff = ['fighter_age_diff', 'height_diff', 'reach_diff']
fight_math_stats = [ '1-fight_math', '6-fight_math', '4-fighter_score_diff', '9-fighter_score_diff', '15-fighter_score_diff' ]
possible_stats = record_statistics + physical_stats + punch_statistics + fight_math_stats
# create the diff features
possible_stats_diff = record_statistics_diff + physical_stats_diff + event_statistics_diff + punch_statistics_diff + fight_math_stats

In [9]:
event_statistics_diff

['fighter_inf_knockdowns_avg_diff_2',
 'fighter_abs_knockdowns_avg_diff_2',
 'fighter_inf_reversals_avg_diff_2',
 'fighter_abs_reversals_avg_diff_2',
 'fighter_inf_sub_attempts_avg_diff_2',
 'fighter_abs_sub_attempts_avg_diff_2']

In [10]:
# LR model score with just record statistics considered
model_score(ufc_fights_winner,record_statistics)

np.float64(-0.6804696396386394)

In [11]:
# LR model score with just record statistics differences considered (higher)
model_score(ufc_fights_winner,record_statistics_diff)

np.float64(-0.6767567207283657)

In [12]:
model_score(ufc_fights_winner,physical_stats)

np.float64(-0.6700850835783903)

In [13]:
model_score(ufc_fights_winner,physical_stats_diff)

np.float64(-0.6694370950326433)

In [14]:
model_score(ufc_fights_winner,punch_statistics)

np.float64(-0.6950046019158217)

In [15]:
model_score(ufc_fights_winner,punch_statistics_diff)

np.float64(-0.6784075416546211)

In [16]:
model_score(ufc_fights_winner,possible_stats)

np.float64(-0.6656277184223823)

In [17]:
# PERFORMANCE IS BETTER WITHOUT SCALING...?
model_score(ufc_fights_winner,possible_stats, scaled=False, _max_iter=5000)

np.float64(-0.6652085822952429)

In [18]:
# GOAL beat this score with a smaller subset of features
# TODO compute the analogous thing by using the vegas odds from bookies (remove vig and compute the implied probability)
# WE NEED TO BEAT THIS SCORE FROM VEGAS
model_score(ufc_fights_winner,possible_stats_diff)

np.float64(-0.6487133411315633)

# NOTE A LARGER VALUE IS BETTER FOR NEG LOG LOSS

In [19]:
print('Length of record_statistics_diff:', len(record_statistics_diff))
print('Length of physical_stats_diff:', len(physical_stats_diff))
print('Length of punch_statistics_diff:', len(punch_statistics_diff))
print('Length of fight_math_stats:', len(fight_math_stats))

Length of record_statistics_diff: 18
Length of physical_stats_diff: 3
Length of punch_statistics_diff: 36
Length of fight_math_stats: 5


In [10]:
# compute 5 choose 2
from math import comb
print('36 choose 4:', comb(36, 4))
print('31 choose 12:', comb(31, 12))

36 choose 4: 58905
31 choose 12: 141120525


In [20]:
record_stats_score_dict = {}
physical_stats_score_dict = {}
event_stats_score_dict = {}
punch_stats_score_dict = {}
fight_math_stats_score_dict = {}

In [21]:
greedy_record_stats, score = greedy(ufc_fights_winner,record_statistics_diff,subsetsize=2,scaled=False)

best subset: ('fighter_L5Y_wins_diff_2', 'fighter_losses_diff_2')
with score -0.6781339683891466


In [22]:
for subsetsize in [1,2,3,4,5]:
    greedy_record_stats, score = greedy(ufc_fights_winner,record_statistics_diff,subsetsize=subsetsize)
    record_stats_score_dict[tuple(greedy_record_stats)] = score

best subset: ('fighter_losses_diff_2',)
with score -0.6859758008480181
best subset: ('fighter_L5Y_wins_diff_2', 'fighter_losses_diff_2')
with score -0.6781204676448449
best subset: ('fighter_L5Y_wins_diff_2', 'fighter_losses_diff_2', 'fighter_ko_losses_diff_2')
with score -0.6760827361018713
best subset: ('fighter_L5Y_wins_diff_2', 'fighter_losses_diff_2', 'fighter_ko_losses_diff_2', 'fighter_sub_wins_diff_2')
with score -0.6755559378759419
best subset: ('fighter_L5Y_wins_diff_2', 'fighter_sub_wins_diff_2', 'fighter_ko_losses_diff_2', 'fighter_losses_diff_2', 'fighter_wins_diff_2')
with score -0.6753187334859281


In [23]:
for subsetsize in [1,2,3]:
    greedy_physical_stats, score = greedy(ufc_fights_winner,physical_stats_diff,subsetsize=subsetsize)
    physical_stats_score_dict[tuple(greedy_physical_stats)] = score

best subset: ('fighter_age_diff',)
with score -0.6702331742089214
best subset: ('reach_diff', 'fighter_age_diff')
with score -0.6696022622640263
best subset: ('reach_diff', 'fighter_age_diff', 'height_diff')
with score -0.6694370950326433


In [24]:
for subsetsize in [1,2,3]:
    greedy_event_stats, score = greedy(ufc_fights_winner,event_statistics_diff,subsetsize=subsetsize)
    event_stats_score_dict[tuple(greedy_event_stats)] = score

best subset: ('fighter_abs_knockdowns_avg_diff_2',)
with score -0.6927119372563671
best subset: ('fighter_inf_knockdowns_avg_diff_2', 'fighter_abs_knockdowns_avg_diff_2')
with score -0.6923702525585034
best subset: ('fighter_abs_reversals_avg_diff_2', 'fighter_inf_knockdowns_avg_diff_2', 'fighter_abs_knockdowns_avg_diff_2')
with score -0.6922544435697604


In [25]:
for subsetsize in [1,2,3,4]:
    greedy_punch_stats, score = greedy(ufc_fights_winner,punch_statistics_diff,subsetsize=subsetsize)
    punch_stats_score_dict[tuple(greedy_punch_stats)] = score

best subset: ('fighter_abs_total_strikes_landed_avg_diff_2',)
with score -0.684001133717203
best subset: ('fighter_inf_takedowns_attempts_avg_diff_2', 'fighter_abs_total_strikes_landed_avg_diff_2')
with score -0.6806733439392402
best subset: ('fighter_inf_takedowns_attempts_avg_diff_2', 'fighter_inf_sig_strikes_landed_avg_diff_2', 'fighter_abs_total_strikes_landed_avg_diff_2')
with score -0.6772647905928239
best subset: ('fighter_abs_takedowns_attempts_avg_diff_2', 'fighter_inf_takedowns_attempts_avg_diff_2', 'fighter_inf_sig_strikes_landed_avg_diff_2', 'fighter_abs_total_strikes_landed_avg_diff_2')
with score -0.6753646869792842


In [26]:
for subsetsize in [1,2,3,4,5,6,7,8,9]:
    greedy_punch_stats, score = greedy(ufc_fights_winner, punch_statistics_diff_set_of_sets, subsetsize=subsetsize, set_of_sets=True)
    punch_stats_score_dict[tuple(greedy_punch_stats)] = score

best subset: (('fighter_inf_total_strikes_landed_avg_diff_2', 'fighter_inf_total_strikes_attempts_avg_diff_2', 'fighter_abs_total_strikes_landed_avg_diff_2', 'fighter_abs_total_strikes_attempts_avg_diff_2'),)
with score -0.6826697861796612
best subset: (('fighter_inf_takedowns_landed_avg_diff_2', 'fighter_inf_takedowns_attempts_avg_diff_2', 'fighter_abs_takedowns_landed_avg_diff_2', 'fighter_abs_takedowns_attempts_avg_diff_2'), ('fighter_inf_total_strikes_landed_avg_diff_2', 'fighter_inf_total_strikes_attempts_avg_diff_2', 'fighter_abs_total_strikes_landed_avg_diff_2', 'fighter_abs_total_strikes_attempts_avg_diff_2'))
with score -0.67848060728802
best subset: (('fighter_inf_takedowns_landed_avg_diff_2', 'fighter_inf_takedowns_attempts_avg_diff_2', 'fighter_abs_takedowns_landed_avg_diff_2', 'fighter_abs_takedowns_attempts_avg_diff_2'), ('fighter_inf_sig_strikes_landed_avg_diff_2', 'fighter_inf_sig_strikes_attempts_avg_diff_2', 'fighter_abs_sig_strikes_landed_avg_diff_2', 'fighter_abs_si

In [27]:
for subsetsize in [1,2,3,4,5]:
    greedy_fight_math_stats, score = greedy(ufc_fights_winner, fight_math_stats, subsetsize=subsetsize)
    fight_math_stats_score_dict[tuple(greedy_fight_math_stats)] = score

best subset: ('4-fighter_score_diff',)
with score -0.6736105965822939
best subset: ('4-fighter_score_diff', '6-fight_math')
with score -0.6723386689925168
best subset: ('4-fighter_score_diff', '6-fight_math', '15-fighter_score_diff')
with score -0.6722794100887622
best subset: ('9-fighter_score_diff', '4-fighter_score_diff', '6-fight_math', '15-fighter_score_diff')
with score -0.6716380530481842
best subset: ('6-fight_math', '1-fight_math', '4-fighter_score_diff', '9-fighter_score_diff', '15-fighter_score_diff')
with score -0.6718273617594558


In [ ]:
# put all the results of the separate greedy runs into a single list
best_stats_from_all_greedy_put_together = []
best_record_stats = max(record_stats_score_dict, key=record_stats_score_dict.get)
best_physical_stats = max(physical_stats_score_dict, key=physical_stats_score_dict.get)
best_event_stats = max(event_stats_score_dict, key=event_stats_score_dict.get)
best_punch_stats = max(punch_stats_score_dict, key=punch_stats_score_dict.get)
best_fight_math_stats = max(fight_math_stats_score_dict, key=fight_math_stats_score_dict.get)
best_stats_from_all_greedy_put_together.extend(best_record_stats)
best_stats_from_all_greedy_put_together.extend(best_physical_stats)
best_stats_from_all_greedy_put_together.extend(best_event_stats)
best_stats_from_all_greedy_put_together.extend(best_punch_stats)
best_stats_from_all_greedy_put_together.extend(best_fight_math_stats)
for stat in best_stats_from_all_greedy_put_together:
    print(stat)


fighter_L5Y_wins_diff_2
fighter_sub_wins_diff_2
fighter_ko_losses_diff_2
fighter_losses_diff_2
fighter_wins_diff_2
reach_diff
fighter_age_diff
height_diff
fighter_abs_reversals_avg_diff_2
fighter_inf_knockdowns_avg_diff_2
fighter_abs_knockdowns_avg_diff_2
fighter_inf_distance_strikes_landed_avg_diff_2
fighter_inf_distance_strikes_attempts_avg_diff_2
fighter_abs_distance_strikes_landed_avg_diff_2
fighter_abs_distance_strikes_attempts_avg_diff_2
fighter_inf_takedowns_landed_avg_diff_2
fighter_inf_takedowns_attempts_avg_diff_2
fighter_abs_takedowns_landed_avg_diff_2
fighter_abs_takedowns_attempts_avg_diff_2
fighter_inf_clinch_strikes_landed_avg_diff_2
fighter_inf_clinch_strikes_attempts_avg_diff_2
fighter_abs_clinch_strikes_landed_avg_diff_2
fighter_abs_clinch_strikes_attempts_avg_diff_2
fighter_inf_head_strikes_landed_avg_diff_2
fighter_inf_head_strikes_attempts_avg_diff_2
fighter_abs_head_strikes_landed_avg_diff_2
fighter_abs_head_strikes_attempts_avg_diff_2
9-fighter_score_diff
4-fight

In [6]:
# result of the above cell (if you dont want to run it all again)

best_stats_from_all_greedy_put_together = [
'fighter_L5Y_wins_diff_2',
'fighter_sub_wins_diff_2',
'fighter_ko_losses_diff_2',
'fighter_losses_diff_2',
'fighter_wins_diff_2',
'reach_diff',
'fighter_age_diff',
'height_diff',
# 'fighter_abs_reversals_avg_diff_2',
'fighter_inf_knockdowns_avg_diff_2',
'fighter_abs_knockdowns_avg_diff_2',
'fighter_inf_distance_strikes_landed_avg_diff_2',
'fighter_inf_distance_strikes_attempts_avg_diff_2',
'fighter_abs_distance_strikes_landed_avg_diff_2',
'fighter_abs_distance_strikes_attempts_avg_diff_2',
'fighter_inf_takedowns_landed_avg_diff_2',
'fighter_inf_takedowns_attempts_avg_diff_2',
'fighter_abs_takedowns_landed_avg_diff_2',
'fighter_abs_takedowns_attempts_avg_diff_2',
'fighter_inf_clinch_strikes_landed_avg_diff_2',
'fighter_inf_clinch_strikes_attempts_avg_diff_2',
'fighter_abs_clinch_strikes_landed_avg_diff_2',
'fighter_abs_clinch_strikes_attempts_avg_diff_2',
'fighter_inf_head_strikes_landed_avg_diff_2',
'fighter_inf_head_strikes_attempts_avg_diff_2',
'fighter_abs_head_strikes_landed_avg_diff_2',
'fighter_abs_head_strikes_attempts_avg_diff_2',
'9-fighter_score_diff',
'4-fighter_score_diff',
'6-fight_math',
'15-fighter_score_diff',
]

In [7]:
model_score(ufc_fights_winner, best_stats_from_all_greedy_put_together, _max_iter=5000)

np.float64(-0.6450672032513766)

In [10]:
len(best_stats_from_all_greedy_put_together)

30

In [ ]:
# remove one at a time until it stops going up
best_subset = best_stats_from_all_greedy_put_together
last_best_score = - 1000
current_best_score = model_score(ufc_fights_winner, best_subset, _max_iter=5000)
while current_best_score > last_best_score:
    last_best_score = current_best_score
    greedy_fight_math_stats, current_best_score = greedy(ufc_fights_winner, best_subset, subsetsize=len(best_subset)-1)
    if current_best_score > last_best_score:
        best_subset = greedy_fight_math_stats
        print(f'Current best score: {current_best_score}')
        print(f'Current best subset: {best_subset}')
    else:
        print(f'No improvement found, stopping greedy search.')
        break

best subset: ('fighter_inf_takedowns_landed_avg_diff_2', 'fighter_abs_head_strikes_landed_avg_diff_2', 'fighter_age_diff', 'fighter_inf_distance_strikes_attempts_avg_diff_2', 'fighter_inf_head_strikes_landed_avg_diff_2', 'fighter_abs_takedowns_landed_avg_diff_2', 'fighter_L5Y_wins_diff_2', 'fighter_abs_reversals_avg_diff_2', 'fighter_abs_takedowns_attempts_avg_diff_2', 'fighter_ko_losses_diff_2', 'fighter_inf_distance_strikes_landed_avg_diff_2', 'height_diff', 'fighter_abs_distance_strikes_attempts_avg_diff_2', 'fighter_inf_clinch_strikes_landed_avg_diff_2', '15-fighter_score_diff', 'fighter_inf_head_strikes_attempts_avg_diff_2', 'fighter_inf_clinch_strikes_attempts_avg_diff_2', '6-fight_math', 'fighter_wins_diff_2', 'fighter_inf_knockdowns_avg_diff_2', 'fighter_losses_diff_2', 'fighter_abs_clinch_strikes_attempts_avg_diff_2', 'fighter_abs_clinch_strikes_landed_avg_diff_2', 'reach_diff', 'fighter_inf_takedowns_attempts_avg_diff_2', 'fighter_sub_wins_diff_2', '9-fighter_score_diff', '4-

In [8]:
# this is the best subset found by removing one feature at a time from the best subset found by the greedy algorithm
best_subset_after_removals = ['fighter_age_diff',
 'fighter_inf_distance_strikes_attempts_avg_diff_2',
 'fighter_L5Y_wins_diff_2',
 'fighter_abs_takedowns_attempts_avg_diff_2',
 'fighter_ko_losses_diff_2',
 'fighter_inf_distance_strikes_landed_avg_diff_2',
 'height_diff',
 'fighter_abs_distance_strikes_attempts_avg_diff_2',
 'fighter_inf_head_strikes_attempts_avg_diff_2',
 '6-fight_math',
 'fighter_wins_diff_2',
 'fighter_inf_knockdowns_avg_diff_2',
 'fighter_losses_diff_2',
 'fighter_abs_clinch_strikes_landed_avg_diff_2',
 'reach_diff',
 'fighter_inf_takedowns_attempts_avg_diff_2',
 'fighter_sub_wins_diff_2',
 '9-fighter_score_diff',
 '4-fighter_score_diff',
 'fighter_abs_head_strikes_attempts_avg_diff_2',
 'fighter_abs_distance_strikes_landed_avg_diff_2']

model_score(ufc_fights_winner, best_subset_after_removals, _max_iter=5000)

np.float64(-0.6430218034958761)

In [28]:
len(best_subset_after_removals)

21

In [32]:
print('36 choose 4:', comb(36, 4))
print('31 choose 14:', comb(21, 14))

36 choose 4: 58905
31 choose 14: 116280


In [ ]:
best_subset_after_removals_subsets_dict = {}
for subsetsize in [14, 15, 16, 17, 18, 19, 20]:
    greedy_stats, score = greedy(ufc_fights_winner, best_subset_after_removals, subsetsize=subsetsize)
    best_subset_after_removals_subsets_dict[tuple(greedy_stats)] = score
# find the max score in the dict 
current_best_subset = max(best_subset_after_removals_subsets_dict, key=best_subset_after_removals_subsets_dict.get)
max_score = best_subset_after_removals_subsets_dict[current_best_subset]
print(f'Max score: {max_score}')
print(f'Max key: {current_best_subset}')

best subset: ('fighter_inf_distance_strikes_landed_avg_diff_2', 'fighter_L5Y_wins_diff_2', 'reach_diff', 'fighter_abs_distance_strikes_attempts_avg_diff_2', 'fighter_inf_takedowns_attempts_avg_diff_2', '6-fight_math', '4-fighter_score_diff', 'fighter_abs_takedowns_attempts_avg_diff_2', '9-fighter_score_diff', 'fighter_age_diff', 'fighter_inf_distance_strikes_attempts_avg_diff_2', 'fighter_abs_head_strikes_attempts_avg_diff_2', 'fighter_abs_distance_strikes_landed_avg_diff_2', 'fighter_ko_losses_diff_2')
with score -0.6391885569950418
best subset: ('fighter_inf_distance_strikes_landed_avg_diff_2', 'fighter_abs_head_strikes_attempts_avg_diff_2', 'fighter_L5Y_wins_diff_2', 'reach_diff', 'fighter_abs_distance_strikes_attempts_avg_diff_2', 'fighter_inf_takedowns_attempts_avg_diff_2', '6-fight_math', '4-fighter_score_diff', 'fighter_abs_takedowns_attempts_avg_diff_2', 'fighter_sub_wins_diff_2', 'fighter_age_diff', 'fighter_inf_distance_strikes_attempts_avg_diff_2', '9-fighter_score_diff', 'f

In [9]:
# after all that hard work, we store the result in a variable
current_best_subset = ['fighter_age_diff', 'fighter_inf_distance_strikes_attempts_avg_diff_2', 'fighter_L5Y_wins_diff_2', 'fighter_abs_takedowns_attempts_avg_diff_2', 'fighter_ko_losses_diff_2', 'fighter_inf_distance_strikes_landed_avg_diff_2', 'height_diff', 'fighter_abs_distance_strikes_attempts_avg_diff_2', 'fighter_inf_head_strikes_attempts_avg_diff_2', '6-fight_math', 'fighter_inf_knockdowns_avg_diff_2', 'fighter_abs_clinch_strikes_landed_avg_diff_2', 'reach_diff', 'fighter_inf_takedowns_attempts_avg_diff_2', 'fighter_sub_wins_diff_2', '9-fighter_score_diff', '4-fighter_score_diff', 'fighter_abs_head_strikes_attempts_avg_diff_2', 'fighter_abs_distance_strikes_landed_avg_diff_2']

In [10]:
best_model_score = model_score(ufc_fights_winner, current_best_subset, _max_iter=5000)
print(f'Best model score: {best_model_score}')
print(f'Probability to observe data given the model: {np.exp(best_model_score)}')

Best model score: -0.6425495803571547
Probability to observe data given the model: 0.5259497619759637


In [11]:
model_score(ufc_fights_winner, current_best_subset, _max_iter=5000, scoring='accuracy')

np.float64(0.641875)

In [41]:
# now start adding remaining features back in one by one or in groups of two or three and see if we can improve the score
current_model_score = best_model_score 
current_best_feature_set = current_best_subset
while True:
    unused_features = set(possible_stats_diff) - set(current_best_feature_set)
    # first try adding one feature 
    single_feature_scores = {}
    for feature in unused_features:
        new_feature_set = current_best_feature_set + [feature]
        new_model_score = model_score(ufc_fights_winner, new_feature_set, _max_iter=5000)
        single_feature_scores[tuple(new_feature_set)] = new_model_score
    best_single_feature_set = max(single_feature_scores, key=single_feature_scores.get)
    best_single_feature_score = single_feature_scores[best_single_feature_set]
    if best_single_feature_score > current_model_score:
        current_best_feature_set = list(best_single_feature_set)
        current_model_score = best_single_feature_score
        print(f'Added single feature: {best_single_feature_set[-1]}')
        print(f'New model score: {current_model_score}')
        continue
    else:
        print('No improvement found with single addition, trying pairs.')
    double_feature_scores = {}
    for feature1, feature2 in itertools.combinations(unused_features, 2):
        new_feature_set = current_best_feature_set + [feature1, feature2]
        new_model_score = model_score(ufc_fights_winner, new_feature_set, _max_iter=5000)
        double_feature_scores[tuple(new_feature_set)] = new_model_score
    best_double_feature_set = max(double_feature_scores, key=double_feature_scores.get)
    best_double_feature_score = double_feature_scores[best_double_feature_set]
    if best_double_feature_score > current_model_score:
        current_best_feature_set = list(best_double_feature_set)
        current_model_score = best_double_feature_score
        print(f'Added double feature: {best_double_feature_set[-2]}, {best_double_feature_set[-1]}')
        print(f'New model score: {current_model_score}')
        continue
    else:
        print('No improvement found with double addition, stopping.')
        break
    

Added single feature: fighter_inf_ground_strikes_landed_avg_diff_2
New model score: -0.6381410699034317
Added single feature: fighter_abs_total_strikes_attempts_avg_diff_2
New model score: -0.6378450099142701
Added single feature: fighter_L5Y_losses_diff_2
New model score: -0.6374232753554653
Added single feature: fighter_L2Y_sub_losses_diff_2
New model score: -0.6370883303848901
Added single feature: fighter_abs_leg_strikes_landed_avg_diff_2
New model score: -0.6369391456699537
Added single feature: fighter_abs_reversals_avg_diff_2
New model score: -0.6369171274939823
Added single feature: fighter_L5Y_ko_wins_diff_2
New model score: -0.6368956247112783
No improvement found with single addition, trying pairs.
Added double feature: fighter_L2Y_ko_losses_diff_2, fighter_L5Y_ko_losses_diff_2
New model score: -0.6368042229873796
No improvement found with single addition, trying pairs.
No improvement found with double addition, stopping.


In [12]:
# storying the result
current_best_feature_set = ['fighter_age_diff',
 'fighter_inf_distance_strikes_attempts_avg_diff_2',
 'fighter_L5Y_wins_diff_2',
 'fighter_abs_takedowns_attempts_avg_diff_2',
 'fighter_ko_losses_diff_2',
 'fighter_inf_distance_strikes_landed_avg_diff_2',
 'height_diff',
 'fighter_abs_distance_strikes_attempts_avg_diff_2',
 'fighter_inf_head_strikes_attempts_avg_diff_2',
 '6-fight_math',
 'fighter_inf_knockdowns_avg_diff_2',
 'fighter_abs_clinch_strikes_landed_avg_diff_2',
 'reach_diff',
 'fighter_inf_takedowns_attempts_avg_diff_2',
 'fighter_sub_wins_diff_2',
 '9-fighter_score_diff',
 '4-fighter_score_diff',
 'fighter_abs_head_strikes_attempts_avg_diff_2',
 'fighter_abs_distance_strikes_landed_avg_diff_2',
 'fighter_inf_ground_strikes_landed_avg_diff_2',
 'fighter_abs_total_strikes_attempts_avg_diff_2',
 'fighter_L5Y_losses_diff_2',
 'fighter_L2Y_sub_losses_diff_2',
 'fighter_abs_leg_strikes_landed_avg_diff_2',
 'fighter_abs_reversals_avg_diff_2',
 'fighter_L5Y_ko_wins_diff_2',
 'fighter_L2Y_ko_losses_diff_2',
 'fighter_L5Y_ko_losses_diff_2']

In [13]:
model_score(ufc_fights_winner, current_best_feature_set, _max_iter=5000)

np.float64(-0.6414052913405859)

In [14]:
model_score(ufc_fights_winner, current_best_feature_set, _max_iter=5000, scoring='accuracy')

np.float64(0.6356250000000001)

In [15]:
current_best_feature_set = [
         'fighter_age_diff',
            'height_diff',
            'reach_diff',
            'fighter_L5Y_wins_diff_2',
            'fighter_L5Y_losses_diff_2',
            'fighter_ko_losses_diff_2',
            'fighter_L5Y_ko_wins_diff_2',
            'fighter_L2Y_ko_losses_diff_2',
            'fighter_L5Y_ko_losses_diff_2',
            'fighter_sub_wins_diff_2',
            'fighter_L2Y_sub_losses_diff_2',
            'fighter_inf_knockdowns_avg_diff_2',
            'fighter_inf_distance_strikes_landed_avg_diff_2',
            'fighter_inf_ground_strikes_landed_avg_diff_2',
            'fighter_inf_distance_strikes_attempts_avg_diff_2',
            'fighter_inf_head_strikes_attempts_avg_diff_2',
            'fighter_inf_takedowns_attempts_avg_diff_2',
            # 'fighter_abs_reversals_avg_diff_2', # reversals has problems
            'fighter_abs_clinch_strikes_landed_avg_diff_2',
            'fighter_abs_distance_strikes_landed_avg_diff_2',
            'fighter_abs_leg_strikes_landed_avg_diff_2',
            'fighter_abs_takedowns_attempts_avg_diff_2',
            'fighter_abs_distance_strikes_attempts_avg_diff_2',
            'fighter_abs_head_strikes_attempts_avg_diff_2',
            'fighter_abs_total_strikes_attempts_avg_diff_2',
            '6-fight_math',
            '9-fighter_score_diff',
            '4-fighter_score_diff',
 ]

y=ufc_fights_winner['result'].iloc[0:40*75]
X=ufc_fights_winner[current_best_feature_set].iloc[0:40*75]
winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=5000, fit_intercept=False)
winPredictionModel.fit(X,y)

print('model score: '+str(winPredictionModel.score(X,y)))
print('neg log loss of model: '+str(model_score(ufc_fights_winner,current_best_feature_set,scoring='neg_log_loss')))
print('cross val avg probability of result: '+str(np.exp(model_score(ufc_fights_winner,current_best_feature_set,scoring='neg_log_loss'))))
print('cross val accuracy: '+str(model_score(ufc_fights_winner,current_best_feature_set,scoring='accuracy')))

model score: 0.6496666666666666
neg log loss of model: -0.6414423217716347
cross val avg probability of result: 0.5265324468973727
cross val accuracy: 0.635625


# WHY IS THE SCORE DIFFERENT THAN FROM ABOVE???

In [64]:
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    # TODO rescale the features to zero mean and unit variance
    
    # return [fighter_score_diff(fighter1,fighter2,day1, 4),
    #         fighter_score_diff(fighter1,fighter2,day1, 9),
    #         fighter_score_diff(fighter1,fighter2,day1, 15),
    #         fight_math_diff(fighter1,fighter2,day1, 1),
    #         fight_math_diff(fighter1,fighter2,day1, 6),
    #         L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
    #         L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
    #         L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
    #         fighter_age(fighter1,day1)-fighter_age(fighter2,day2),
    #         avg_count('total_strikes_landed',fighter1,'abs',day1)-avg_count('total_strikes_landed',fighter2,'abs',day2),
    #         avg_count('head_strikes_landed',fighter1,'abs',day1)-avg_count('head_strikes_landed',fighter2,'abs',day2),
    #         avg_count('ground_strikes_landed',fighter1,'inf',day1)-avg_count('ground_strikes_landed',fighter2,'inf',day2),
    #         avg_count('takedowns_attempts',fighter1,'inf',day1)-avg_count('takedowns_attempts',fighter2,'inf',day2),
    #         avg_count('head_strikes_landed',fighter1,'inf',day1)-avg_count('head_strikes_landed',fighter2,'inf',day2),
    #     ]
    return [
    # 'fighter_age_diff',
    #     'height_diff',
    #     'reach_diff',
    #     'fighter_L5Y_wins_diff_2',
    #     'fighter_L5Y_losses_diff_2',
    #     'fighter_ko_losses_diff_2',
    #     'fighter_L5Y_ko_wins_diff_2',
    #     'fighter_L2Y_ko_losses_diff_2',
        fighter_age(fighter1, day1) - fighter_age(fighter2, day2),
        fighter_height(fighter1) - fighter_height(fighter2),
        fighter_reach(fighter1) - fighter_reach(fighter2),
        L5Y_wins(fighter1, day1) - L5Y_wins(fighter2, day2),
        L5Y_losses(fighter1, day1) - L5Y_losses(fighter2, day2),
        ko_losses(fighter1, day1) - ko_losses(fighter2, day2),
        L5Y_ko_wins(fighter1, day1) - L5Y_ko_wins(fighter2, day2),
        L2Y_ko_losses(fighter1, day1) - L2Y_ko_losses(fighter2, day2),
        #     'fighter_L5Y_ko_losses_diff_2',
    #     'fighter_sub_wins_diff_2',
    #     'fighter_L2Y_sub_losses_diff_2',
    #     'fighter_inf_knockdowns_avg_diff_2',
        L2Y_ko_losses(fighter1, day1) - L2Y_ko_losses(fighter2, day2),
        sub_wins(fighter1, day1) - sub_wins(fighter2, day2),
        L2Y_sub_losses(fighter1, day1) - L2Y_sub_losses(fighter2, day2),
        avg_count('knockdowns', fighter1, 'inf', day1) - avg_count('knockdowns', fighter2, 'inf', day2),
    
    #     'fighter_inf_distance_strikes_landed_avg_diff_2',
        avg_count('distance_strikes_landed', fighter1, 'inf', day1) - avg_count('distance_strikes_landed', fighter2, 'inf', day2),
    #     'fighter_inf_ground_strikes_landed_avg_diff_2',
        avg_count('ground_strikes_landed', fighter1, 'inf', day1) - avg_count('ground_strikes_landed', fighter2, 'inf', day2),
    #     'fighter_inf_distance_strikes_attempts_avg_diff_2',
        avg_count('distance_strikes_attempts', fighter1, 'inf', day1) - avg_count('distance_strikes_attempts', fighter2, 'inf', day2),
        
        
        #     'fighter_inf_head_strikes_attempts_avg_diff_2',
        avg_count('head_strikes_attempts', fighter1, 'inf', day1) - avg_count('head_strikes_attempts', fighter2, 'inf', day2),
    #     'fighter_inf_takedowns_attempts_avg_diff_2',
        avg_count('takedowns_attempts', fighter1, 'inf', day1) - avg_count('takedowns_attempts', fighter2, 'inf', day2),
    #     'fighter_abs_reversals_avg_diff_2',
        # avg_count('reversals', fighter1, 'abs', day1) - avg_count('reversals', fighter2, 'abs', day2),
    #     'fighter_abs_clinch_strikes_landed_avg_diff_2',
        avg_count('clinch_strikes_landed', fighter1, 'abs', day1) - avg_count('clinch_strikes_landed', fighter2, 'abs', day2),
    #     'fighter_abs_distance_strikes_landed_avg_diff_2',
        avg_count('distance_strikes_landed', fighter1, 'abs', day1) - avg_count('distance_strikes_landed', fighter2, 'abs', day2),
        
            #     'fighter_abs_leg_strikes_landed_avg_diff_2',
        avg_count('leg_strikes_landed', fighter1, 'abs', day1) - avg_count('leg_strikes_landed', fighter2, 'abs', day2),
    #     'fighter_abs_takedowns_attempts_avg_diff_2',
        avg_count('takedowns_attempts', fighter1, 'abs', day1) - avg_count('takedowns_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_distance_strikes_attempts_avg_diff_2',
        avg_count('distance_strikes_attempts', fighter1, 'abs', day1) - avg_count('distance_strikes_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_head_strikes_attempts_avg_diff_2',
        avg_count('head_strikes_attempts', fighter1, 'abs', day1) - avg_count('head_strikes_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_total_strikes_attempts_avg_diff_2',
        avg_count('total_strikes_attempts', fighter1, 'abs', day1) - avg_count('total_strikes_attempts', fighter2, 'abs', day2),
                #     '6-fight_math',
    #     '9-fighter_score_diff',
    #     '4-fighter_score_diff',
        fight_math_diff(fighter1, fighter2, day1, 6),
        fighter_score_diff(fighter1, fighter2, day1, 9),
        fighter_score_diff(fighter1, fighter2, day1, 4),
    ]
    

In [65]:
dataframe = ufc_fights_winner
iloc_val = 3200
_max_iter = 2000
scoring='neg_log_loss'
scaled=True

yyy=dataframe['result'].iloc[0:iloc_val]
XXX=dataframe[current_best_feature_set].iloc[0:iloc_val]
XXXscaler = preprocessing.StandardScaler().fit(XXX)
XXX_scaled = XXXscaler.transform(XXX) 
X = XXX_scaled if scaled else XXX
winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=_max_iter, fit_intercept=False)
# find the cross val score with log loss
cross_val_score(winPredictionModel,X,yyy,cv=4,scoring=scoring).mean()

np.float64(-0.636851776860466)

In [59]:
tup = ufc_prediction_tuple('Cody Garbrandt', 'Pedro Munhoz', 'July 5, 2023', 'July 5, 2023')

In [67]:
tup_scaled = XXXscaler.transform(np.array(tup).reshape(1, -1))

c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [69]:
tup_scaled[0]

array([-9.39214737e-01,  5.25232669e-01,  6.44039497e-02, -1.04297667e+00,
        2.23999702e-03,  2.60503232e+00, -5.59192401e-01,  1.30968888e+00,
        8.20152356e-01, -1.35430404e+00,  3.25929778e-02,  1.92329836e-01,
       -1.07448769e+00, -4.44964215e-02, -1.02802550e+00, -4.62174005e-01,
        1.49467900e-01,  1.43395072e-01, -1.16362433e+00, -1.92815795e-01,
       -5.85468494e-01, -9.69663487e-01, -7.98516289e-01, -8.64847548e-01,
       -2.57155221e+00, -1.23207639e+00, -7.88462544e-01])